In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import numpy as np
import cupy as cp
import copy
import pandas as pd
import glob

local


In [2]:
def find_legalized(N_trees, save_result=True):
    try: 
        import kaggle_support as kgs
        import pack_io
        import numpy as np
        import os 
        fname = kgs.code_dir + f'../res/sol_legalized_{N_trees}.pickle'
        if os.path.isfile(fname):
            print(f"{N_trees}: Legalized solution already exists. Loading from file.")
            return kgs.dill_load(fname)
        kgs.set_float32(False)
        kgs.TREE_EXPANSION = (20*(1.000001-1))**2
        kgs.initialize_tree_globals()
        xyt_list = kgs.dill_load(kgs.code_dir + '../res/sols.pickle')[N_trees-1]
        sol_list = []
        scores = []
        for xyt in xyt_list:
            sol = kgs.SolutionCollectionSquare()
            sol.xyt = xyt.astype(kgs.dtype_cp)        
            try:
                _,score = pack_io.solution_list_to_dataframe([sol], return_scores= True, print_score=False)
                scores.append(score[0])
            except:
                #print('failed to score solution')
                scores.append(np.nan)                
            sol_list.append(sol)
        #print(scores)
        min_score_before = np.nanmin(scores)
        #_,scores_test = pack_io.solution_list_to_dataframe(sol_list, return_scores= True, print_score=False)
        #print(scores_test)
        sol_list = [sol for sol, score in zip(sol_list, scores) if (np.isnan(score) or score <= min_score_before + 1e-3)]
        #_,scores_test = pack_io.solution_list_to_dataframe(sol_list, return_scores= True, print_score=False)
        #print(scores_test)
        print(f"{N_trees}: Considering {len(sol_list)} solutions")
        sol_list_legalized = []
        for sol in sol_list:
            #pack_io.solution_list_to_dataframe([sol], return_scores= True, print_score=True)
            sol_legalized = pack_io.legalize(sol, do_plot = False, validate=True, target = 1e-20, n_iter = 100, tolerance_rel_change=1e-10, line_search = False, verbose=False, scaling=1.)
            sol_list_legalized.append(sol_legalized)
            #pack_io.solution_list_to_dataframe([sol_legalized], return_scores= True, print_score=True)
        _,scores = pack_io.solution_list_to_dataframe(sol_list_legalized, return_scores= True, print_score=False)
        print(f"{N_trees}: Score before legalize: {min_score_before:.8f}")
        print(f"{N_trees}: Score after legalize: {np.nanmin(scores):.8f}")
        if save_result:
            kgs.dill_save(fname, sol_list_legalized[np.argmin(scores)])
        return sol_list_legalized[np.argmin(scores)]
    except Exception as e:
        print(f"Error in legalizing for N_trees={N_trees}: {e}")
        raise e

In [ ]:
#find_legalized(41)

200: Considering 3 solutions
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved_60701.cu -o /mnt/d//packing/temp/pack_cuda_crystal_60701.cubin
ptxas info    : 144 bytes gmem, 2324 bytes cmem[3], 8 bytes cmem[4]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    88 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 38 registers, used 1 barriers, 88 bytes cumulative stack size, 404 bytes cmem[0], 120 bytes cmem[2]
ptxas info    : Compile time = 21.750 ms
ptxa

SolutionCollectionSquare(comment='', xyt=array([[[-3.44562673,  1.80842645,  1.2497528 ],
        [-2.36851076, -0.21575288,  1.25934493],
        [-1.75150447,  3.18034244,  4.35832163],
        [-3.80658061,  3.53272243,  0.44487308],
        [-3.27471143,  0.19826363,  2.45353453],
        [-0.81661846, -0.76340679,  4.40093759],
        [-3.16149917,  2.33737159,  0.41526424],
        [-3.82394479, -0.41077638,  2.72918221],
        [-3.0611791 , -1.19063981,  1.21795475],
        [-3.51631592, -2.72690943,  4.40742655],
        [-3.14455863,  1.21512575,  4.40078572],
        [-2.62375846, -3.22191022,  4.40742953],
        [-0.24431175,  2.72919596,  1.2643943 ],
        [-2.75184896,  3.64808644,  4.31282521],
        [-3.63709132, -1.22899403,  4.34359006],
        [-1.14669576,  3.22358922,  1.26894033],
        [-1.53436707, -3.64802829,  5.11291061],
        [-2.08260059,  2.71479298,  4.35607596],
        [-0.2749623 , -1.74687537,  4.40639056],
        [-1.47255295, -0.709

In [ ]:
# Run legalize for N=1 to 200
import multiprocess as mp

if __name__ == '__main__':
    # Create a pool of 20 workers
    with mp.Pool(processes=20) as pool:
        # Run find_legalized for N=1 to 200 in parallel
        pool.map(find_legalized, range(1, 201), chunksize=1)
    
    print("All legalizations completed!")

local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
